In [ ]:
import yaml
import pickle
import os.path as osp
import subprocess
from datetime import timedelta
from urllib.parse import urlparse
import numpy as np
import matplotlib.pyplot as plt
from moisture_rnn_pkl import pkl2train
import time
from utils import time_intp, str2time, filter_nan_values, read_pkl, read_yml, retrieve_url

## Setup

In [ ]:
retrieve_url("https://demo.openwfm.org/web/data/fmda/dicts/fmda_nw_202401-05_f05.pkl", "data/fmda_nw_202401-05_f05.pkl")

In [ ]:
data_params = read_yml("params_data.yaml")
data_params

In [ ]:
# dat = read_pkl("data/test_CA_202401.pkl")
dat = read_pkl("data/fmda_nw_202401-05_f05.pkl")

## Format and Filter

In [ ]:
train_h = process_train_dict("data/test_CA_202401.pkl", atm_dict="HRRR", data_params=data_params)

In [ ]:
train_r = process_train_dict("data/test_CA_202401.pkl", atm_dict="RAWS", data_params=data_params)